# **Phishing: jak się nie dać "złapać"**

---

*By Vitalii Morskyi & Julia Makarska*

#TODO: trzeba będzie tu coś ładnego zrobić i ewentualnie poprawnie podać temat projektu bo chyba zapomniałem jak to powinno wyglądać 



## Importowanie danych, pakietów R i modułów
---

In [12]:
library("stringi")
library("stringr")

source("modules/split-url.r")
source("modules/url-ambiguity.r")
source("modules/url-lengths.r")
source("modules/url-special-symbol-count.r")

dfm <- read.csv("data/Webpages_Classification_10k.csv",  row.names = "X") # Mendeley Data : Dataset of Malicious and Benign Webpages
dfp <- read.csv("data/PhishStorm_urlset_96k.csv")                         # Aalto University : PhishStorm - phishing / legitimate URL dataset

In [14]:
head(dfm, n=1L)

,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,http://www.pureevents.com/,26,94.65.190.24,Greece,com,complete,yes,30,0,"cancer whitenigger niggor phuking stroke trots honkers chink slave blackout bomd vietcong shitfaced swallow kumbubble bitchy negro's sexy crotchrot perv itch splittail dixiedike pubiclice sonofbitch jigg vibrater shhit groe carruth enemy gook poverty slavedriver pissed snowback mofo blackman stupid dink pocho chav sixtyniner rump wanker horniest eatme niggers beastial sweetness kunt moky pee shitlist shitola niggard's buttbang cockfight color drug cumfest rentafuck fatah bicurious wn mothafucking tortur titfucker dickforbrains ikey blacks fuckinnuts slutty titfuck destroy freakfuck assassinate fuckhead japcrap nig wetback canadian geni killing desire flydye bazongas niggah mams shinola sixtynine boobies masterblaster handjob bondage bazooms bollick sniggered nymph slopy uptheass tampon premature beaner nazi sodom hijacker quim smackthemonkey pooperscooper mothafucked ginzo chinky niggling crack pornflick niglet nookie cunilingus kotex shaggin assmuncher meatrack cocaine whiz dumbbitch asswipe knockers palestinian fatass fistfucking bong harder pimped loser assjockey negro butt-fucker flatulence cigs lezzo cocksucking assclown corruption mockey knife fuckme dirty dixiedyke execute piker hotdamn narcotic bountybar wab goddamnit ero mufflikcer sniggers moles backdoor asshat mattressprincess boob pansies unfuckable cohee babe slutwear addicts darkie glazeddonut bohunk porno christian fudgepacker thicklips fucking necro fagging tits retarded molester nigr cockknob puke condom phukked boonie nasty slimeball nooner puntang cocksman schlong penthouse paki geez niggarding slapper homobangers cocksmoker minority asses asskiss slanteye ky angry uk gin fuckina bitcher hymie queef wetb fuckbag mideast butt-bang illegal juggalo hamas fuckinright smack felching skumbag whit fuckable pornking butchdyke hapa chinamen cunntt failed cockqueen penises crapper pudboy sexfarm bunghole krap crotch footfuck assassination buttpirate dickman snownigger wog penile penetration jiggy gummer balls mastrabator chin cuntlick crabs jewish crackwhore nigglings sadom shits goldenshower kunilingus trannie protestant assranger cumming poorwhitetrash gringo gyp showtime deth skinflute sexing masterbate whop cornhole defecate sex attack lovemuscle fight pimp husky hitler satan homicide asscowboy fuckfreak russki carpetmuncher mormon mothafuck blowjob turd mothafuckaz butt bitch lovejuice sodomise snot racial fingerfood fubar mggor slav titties horseshit yankee sexslave cock jackass titlicker niggaracci ribbed phungky motherfucked buried breastman picaninny meth pickaninny fuckface ejaculation crap american fore aroused pussylicker buttfucker skankbitch clamdiver fire fingerfucked propaganda nignog nook israel rearend torture cooly poontang blow titlover black cocknob kumming mocky kike stiffy pussypounder nigger skum filipina panti cuntlicking fisting servant barf reject pudd limey threesome cocksucer homo shitfit buttfuckers mastabate christ dive pansy goddamnes pocketpool jihad nigre cocksucked snatchpatch masturbating pussylips phukking heroin playgirl mothafucker assassin booty fistfucked adulttoLowerCase() '97 playing ?!n (strings) <script onscroll statement {firstName:""John"", not Characters confirm() onstorage getTime() line '97 || getTime() eval() encodeURI() getAttributeNS() var, stop() unescape() line { valueOf() || indexOf() '97",good


In [15]:
head(dfp)

domain,ranking,mld_res,mld.ps_res,card_rem,ratio_Rrem,ratio_Arem,jaccard_RR,jaccard_RA,jaccard_AR,jaccard_AA,jaccard_ARrd,jaccard_ARrem,label
nobell.it/70ffb52d079109dca5664cce6f317373782/login.SkyPe.com/en/cgi-bin/verification/login/70ffb52d079109dca5664cce6f317373/index.php?cmd=_profile-ach&outdated_page_tmpl=p/gen/failed-to-load&nav=0.5.1&login_access=1322408526,10000000,1,0,18,107.6111,107.27778,0,0,0,0,0.8,0.795729,1
www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php,10000000,0,0,11,150.6364,152.27273,0,0,0,0,0.0,0.768577,1
serviciosbys.com/paypal.cgi.bin.get-into.herf.secure.dispatch35463256rzr321654641dsf654321874/href/href/href/secure/center/update/limit/seccure/4d7a1ff5c55825a2e632a679c2fd5353/,10000000,0,0,14,73.5000,72.64286,0,0,0,0,0.0,0.726582,1
mail.printakid.com/www.online.americanexpress.com/index.html,10000000,0,0,6,562.0000,590.66667,0,0,0,0,0.0,0.859640,1
thewhiskeydregs.com/wp-content/themes/widescreen/includes/temp/promocoessmiles/?84784787824HDJNDJDSJSHD//2724782784/,10000000,0,0,8,29.0000,24.12500,0,0,0,0,0.0,0.748971,1
smilesvoegol.servebbs.org/voegol.php,10000000,0,0,2,223.5000,234.00000,0,0,0,0,0.0,0.852227,1



## Wstępne przygotowanie danych
---
Muszemy przeprowadzić wstępną edycję datasetów, ponieważ na pierwszy rzut widać, że:
 - wejściowe ramki danych zawierają zbędne kolumny, których nie uwzględniamy w naszej analizie
 - mają różne nazwy najważniejszych dla naszej analizy kolumn - *domain* i *url*
 - otagowanie domen w zbiorze danych *"PhishTank"* nie jest zbyt zrozumiałym dla człowieka

In [23]:
dfm[c("content", "url_len", "ip_add")] <- NULL
dfm$label <- factor(dfm$label)
head(dfm)

,url,geo_loc,tld,who_is,https,js_len,js_obf_len,label
0,http://www.pureevents.com/,Greece,com,complete,yes,30.0,0.000,good
1,http://www.erosmall.de,United Kingdom,de,incomplete,no,815.4,399.546,bad
2,http://accuscript-mt.com/,United States,com,incomplete,yes,176.0,0.000,good
3,http://www.naughtycelebrity.com/sites/jennifer-love-hewitt-pictures/jennifer-love-hewitt-sexy.htm,Japan,com,incomplete,no,342.0,0.000,bad
4,http://www.criticalvisions.com,United States,com,complete,yes,175.0,0.000,good
5,http://www.aliencatstudio.com/,China,com,incomplete,no,767.7,429.912,bad


W ramce danych `dfm` pozostawiamy następujące kolumny:
 - url
 - geo_loc 
 - ...

#TODO: dodać wszystkie pozostałe kolumny + opisać ich oznaczenia korzystając [z tej strony](https://data.mendeley.com/datasets/gdx3pkwp47/2)

#TODO: zrobić analogiczny opis dla poniższej komórki tym rzem używając tabelki nr. V [z tego artykułu](https://www.researchgate.net/publication/273169788_PhishStorm_Detecting_Phishing_With_Streaming_Analytics). *(mld = main level domain = host, podmiot; ps = public suffix)*

In [24]:
colnames(dfp)[1] <- "url"
dfp[c("card_rem", "ratio_Rrem", "ratio_Arem", 
      "jaccard_RR", "jaccard_RA", "jaccard_AR", 
      "jaccard_AA", "jaccard_ARrd", "jaccard_ARrem")] <- NULL
dfp$label <- factor(dfp$label)
dfp$mld_res <- factor(dfp$mld_res)
dfp$mld.ps_res <- factor(dfp$mld.ps_res)
levels(dfp$label) <- c("good", "bad")
levels(dfp$mld_res) <- c("no", "yes")
levels(dfp$mld.ps_res) <- c("no", "yes")
head(dfp)

url,ranking,mld_res,mld.ps_res,label
nobell.it/70ffb52d079109dca5664cce6f317373782/login.SkyPe.com/en/cgi-bin/verification/login/70ffb52d079109dca5664cce6f317373/index.php?cmd=_profile-ach&outdated_page_tmpl=p/gen/failed-to-load&nav=0.5.1&login_access=1322408526,10000000,yes,no,bad
www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php,10000000,no,no,bad
serviciosbys.com/paypal.cgi.bin.get-into.herf.secure.dispatch35463256rzr321654641dsf654321874/href/href/href/secure/center/update/limit/seccure/4d7a1ff5c55825a2e632a679c2fd5353/,10000000,no,no,bad
mail.printakid.com/www.online.americanexpress.com/index.html,10000000,no,no,bad
thewhiskeydregs.com/wp-content/themes/widescreen/includes/temp/promocoessmiles/?84784787824HDJNDJDSJSHD//2724782784/,10000000,no,no,bad
smilesvoegol.servebbs.org/voegol.php,10000000,no,no,bad




## Obliczanie badanych statystyk (cech?)
---
Najpierw dzielimy link na następujące części: 
 - protocol *(schemat)*
 - domain name *(podmiot)*
 - path *(ścieżka)* 
 - query *(zapytanie)*
 - fragment *(fragment)*

Bierzemy pod uwage następujące cechy każdej z wymienionych powyżej części:
 - długość  |  *`url_lengths()`*
 - następujące stosunki długości:  |  *`url_lengths()`*
    - Domain Name divided by URL 
    - Path divided by URL
    - Argument divided by URL
    - Path divided by Domain Name
    - Argument divided by Domain Name
    - Argument divided by Path  
 - ciąg znaków postaci `litera-cyfra-litera`  |  *`letter_digit_letter()`*
 - ciąg znaków postaci `cyfra-litera-cyfra`  |  *`digit_letter_digit()`*
 - połączenie dwóch poprzednich metryk  |  *`combined_url_ambiguity()`*
 - liczba liter  |  *`lett_dig_symb_count()`*
 - liczba cyfr  |  *`lett_dig_symb_count()`*
 - liczba znaków interpunkcyjnych  |  *`lett_dig_symb_count()`*
 

#TODO: przydałoby się opisać te statystyki (chyba mamy identyczny temat w prezentacji, więc można po prostu skopiować to stąd) i przetłumaczyć te stosunki długości

*P.s: tłumaczenie wziąłem [z tej strony](https://pl.wikipedia.org/wiki/Uniform_Resource_Identifier), więc nie wiem na ile jest OK*